# A (Possible) Code for Matching Timepix Clusters to Tagger Hits

First, we'll import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import Timepix
import ROOT
import numpy as np
import pandas as pd
import root_numpy
import json
from collections import OrderedDict
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed and the tagger calibration file (?):

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

aqdet.LoadDetectors(["taggerWithOffsets.json"])

Mk2 Data
taggerWithOffsets.json


Get some tagger info??

In [3]:
taggerChannels = aqdet.detmask["Tagger"]
taggerADC = []

for i in taggerChannels:
    taggerADC += [i[0]]

#taggerADC.sort()

    
print(taggerADC)


    
    
#taggerData = aqdet.Calibrate(aq.adcArray)
#taggerADC = aqdet.Arrays["Tagger"]["Time"][0]
#print(taggerADC)

[849, 832, 833, 848, 817, 800, 801, 816, 977, 960, 961, 976, 945, 928, 929, 944, 1105, 1088, 1089, 1104, 1073, 1056, 1057, 1072, 913, 896, 897, 912, 881, 864, 865, 880, 1041, 1024, 1025, 1040, 1009, 992, 993, 1008, 1169, 1152, 1153, 1168, 1137, 1120, 1121, 1136, 851, 834, 835, 850, 819, 802, 803, 818, 979, 962, 963, 978, 947, 930, 931, 946, 1107, 1090, 1091, 1106, 1075, 1058, 1059, 1074, 915, 898, 899, 914, 883, 866, 867, 882, 1043, 1026, 1027, 1042, 1011, 994, 995, 1010, 1171, 1154, 1155, 1170, 1139, 1122, 1123, 1138, 853, 836, 837, 852, 821, 804, 805, 820, 981, 964, 965, 980, 949, 932, 933, 948, 1109, 1092, 1093, 1108, 1077, 1060, 1061, 1076, 917, 900, 901, 916, 885, 868, 869, 884, 1045, 1028, 1029, 1044, 1013, 996, 997, 1012, 1173, 1156, 1157, 1172, 1141, 1124, 1125, 1140, 855, 838, 839, 854, 823, 806, 807, 822, 983, 966, 967, 982, 951, 934, 935, 950, 1111, 1094, 1095, 1110, 1079, 1062, 1063, 1078, 919, 902, 903, 918, 887, 870, 871, 886, 1047, 1030, 1031, 1046, 1015, 998, 999, 1014,

Set up some histograms:

In [4]:
hist_posA = Hist2D(256,0,256,256,0,256,name='posA')
hist_posACoin = Hist2D(256,0,256,256,0,256,name='posACoin')
hist_timeA = Hist(40000,0,200000000,name='TimeA')
hist_tdiff = Hist((400000+400000)/5000,-400000,400000,name='timeDiffA')

And now, for the fun stuff:

In [6]:
triggerTime = []
scalerRepeat = 0
eventA  = 0    # event number for timepix A
prevTriggerTime = [0]
pulseCount   = 0
taggTime = []

#global counters for cluster info
counter = 0
hits    = 0
group   = -1


sumx = 0.0
sumy = 0.0
meanx = 0.0
meany = 0.0
gn = 1
n = 0.0

def TimepixTagger():
    global triggerTime
    global scalarRepeat
    global eventA
    global prevTriggerTime
    global pulseCount
    global taggTime
    global taggerADC
        
    # ADC value which increases on a pulser
    ADC104    = aq.adcArray[np.where(aq.adcArray['adc']==104)[0]]['val'][0]
    # ADC values which define the clock of the trigger
    shortTime = aq.adcArray[np.where(aq.adcArray['adc']==102)[0]]['val'][0]
    longTime  = aq.adcArray[np.where(aq.adcArray['adc']==103)[0]]['val'][0]
    # Calibration of the clock to roughly ns
    # Add this time to the list of times
    triggerTime += [8*(shortTime+65536*(longTime+65536*scalerRepeat))]
    
   # for i in taggerADC:
   #     adc = aq.adcArray[np.where(aq.adcArray['adc']==i)[0]]['val']
   #     if len(adc)>=1:
   #         print(adc)
    
    #taggTime += [aq.adcArray[np.where(aq.adcArray['adc']==832)[0]]['val'][0]]
    #print(taggTime)
    
    # Included because there was some problem at the start of the file, probably a better way to do it
    if(len(triggerTime)<=1): return

    
#
   #
      #
   #         
# 
    
    # if event contains epics information
    if(aq.epicsEvent==1):
        # check if the epics information has been updated since the previous epics event
        if(eventA!=aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')):
            # Get the raw information out of the epics event
            eventA = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
            nHitsA = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
            encodedA = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            
            # decode the data and create a data frame for timepix A
            TimepixAData = pd.DataFrame(Timepix.Decode(nHitsA,encodedA)) 
            
            # calibrate the times and put them in the data frame
            TimepixAData['ctime']=16*TimepixAData['ToA'].astype(long)-(TimepixAData['FToA'].astype(int))
            
            # sort by the calibrated times
            TimepixAData=TimepixAData.sort_values(by='ctime')
            TimepixAData = TimepixAData.reset_index(drop=True)
            
            # determine the cluster groupings
            TimepixAData['tdiff']=TimepixAData['ctime']-TimepixAData['ctime'].shift() # get the time diffs between hits
            TimepixAData=TimepixAData.fillna(1000).astype(int)    # replace NaNs with 0s
            
            # call function to add new columns with total time of a cluster, number of hits, and group number
            newValues = TimepixAData.apply(find_clusters,axis=1)
            newValues.columns = ['cluster #','nhits','twalk']
            TimepixAData = pd.concat([TimepixAData, newValues], axis=1)
            
            
#
   #
      #
   #         
#       
            # making a data frame where the entries are clusters rather than hits
            ClustersA = TimepixAData.groupby(['cluster #'],as_index=False).mean()
            ClustersA = ClustersA.drop(['ToA','FToA','ToT','tdiff','twalk','nhits', 'ctime'],axis=1)
            ClustersA.rename(columns={'x':'mean x','y':'mean y'}, inplace=True)
            
            nh = []
            tw = []
            ft = [TimepixAData['ctime'][0]]

            g = TimepixAData['cluster #'][0]
            for i in range(len(TimepixAData)):
                if (TimepixAData['cluster #'][i]!=g):
                    nh += [TimepixAData['nhits'][i-1]+1]
                    tw += [TimepixAData['twalk'][i-1]]
                    ft += [TimepixAData['ctime'][i]]
                    g+=1
            if (g==TimepixAData['cluster #'][i]):
                nh += [TimepixAData['nhits'][i]+1]
                tw += [TimepixAData['twalk'][i]]

            ClustersA['nhits'] = nh
            ClustersA['tspan'] = tw
            ClustersA['tfirst'] = ft

            hist_posA.fill_array(ClustersA[['mean x','mean y']])
            #hist_timeA.fill_array(ClustersA['tfirst'])
            


#
   #
      #
   #         
#     
            
            # If the trigger clock has looped add the loop length to the variables
            triggerTime2 = np.array(triggerTime,dtype=np.float)
            triggerWrap  = np.where(np.diff(triggerTime2)<0)[0]
            if(len(triggerWrap)>0):
                triggerTime2[triggerWrap[0]:] += 65536*65536*8
                
            # Determine which pulser trigger relates to the timepix event
            # This seems to get mixed up between the previous one and the one before that
            # Some are still wrong but enough right not to worry for now.
            ## I don't really understand this part
            timeSub = prevTriggerTime[-1]
            if((triggerTime2-timeSub)[-1]<1000000000 and len(prevTriggerTime)>1): 
                timeSub = prevTriggerTime[-2]
            
            # Subtract that previous time
            triggerTime2 -= timeSub
            # Convert the clock times into 1.5625ns.
            # The differnce between the timepix and trigger clocks seems to be by a factor 1.015595 or there abouts
            triggerTime2 /= 1.015595*1.5625
       
            # Coincidence index list
            coinIdx = []
            
            
            
            # Loop through the timepix times finding which ones are in coincidence with trigger events
            for i, hit in enumerate(ClustersA['tfirst']):
                # Difference between the trigger times and the Timepix times
                tdiff = triggerTime2-hit
                
                hist_tdiff.fill_array(tdiff)

                # If there is a coincidence add the event index to the list
                # For some reason there are two peaks...
                # Could have a separate list of random events which could be scaled and subtracted like with the tagger.
                if (len(tdiff[np.abs(tdiff)<10000])>0 or len(tdiff[np.abs(tdiff-263000)<10000])>0):
                    coinIdx +=  [i]
            
            
            # Make arrays just containing coincedent hits with Timepix A    
            coinHit  = ClustersA['tfirst'][coinIdx]
            coinXPos = ClustersA['mean x'][coinIdx]
            coinYPos = ClustersA['mean y'][coinIdx]
            # Fill histogram with positions of coincident events in Timepix A
            hist_posACoin.fill_array(np.vstack([coinXPos,coinYPos]).T)
                
                

            # Update the trigger times for the next buffer
            # A bit more complex than ideal again because it might not be just the previous set of events.
            triggerfilter = prevTriggerTime[-1]
            if(triggerTime[0]>triggerfilter): 
                triggerfilter += 65536*65536*8
            triggerTime     = triggerTime[np.where(triggerTime<=triggerfilter)[0][-1]:]
                
            prevTriggerTime = []
            
            

#
   #
      #
   #         
#                       

            ###








#
   #
      #
   #         
#                       
 
            #print(ClustersA)
            #print(TimepixAData)
            
            #eventA = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')   
            
    if(aq.eventNo%10000==0):                          # print a processing statement every 5000 events
        print("Events Processed: "+str(aq.eventNo))
    if(ADC104!=pulseCount):
        #print pulseCount, ADC104
        pulseCount  = ADC104        
        #print len(triggerTime)
        #print triggerTime[-1]-triggerTime[0]
        #print ' '
        prevTriggerTime += [triggerTime[-1]]
        #triggerTime = []
    
aq.runFunction(TimepixTagger,0,5000000)

Events Processed: 10000
Events Processed: 20000
Events Processed: 30000
Events Processed: 40000
Events Processed: 50000
Events Processed: 60000
Events Processed: 70000
Events Processed: 80000
Events Processed: 90000
Events Processed: 100000
Events Processed: 110000
Events Processed: 120000
Events Processed: 130000
Events Processed: 140000
Events Processed: 150000
Events Processed: 160000
Events Processed: 170000
Events Processed: 180000
Events Processed: 190000
Events Processed: 200000
Events Processed: 210000
Events Processed: 220000
Events Processed: 230000
Events Processed: 240000
Events Processed: 250000
Events Processed: 260000
Events Processed: 270000
Events Processed: 280000
Events Processed: 290000
Events Processed: 300000
Events Processed: 310000
Events Processed: 320000
Events Processed: 330000
Events Processed: 340000
Events Processed: 350000
Events Processed: 360000
Events Processed: 370000
Events Processed: 380000
Events Processed: 390000
Events Processed: 400000
Events Pr

ValueError: Length mismatch: Expected axis has 7 elements, new values have 3 elements

In [5]:
# this gets applied to every row in the dataframe input, and determines which cluster that hit belongs to
def find_clusters(data):
    global counter    # time from the first hit in the counter
    global hits       # number of hits in the cluster
    global group      # allocate seperate group numbers for each cluster
    
    if (data['tdiff'] > 100):   # if too far, start new cluster
        counter = 0            # zero the time counter
        hits = 0               # zero the hits for the cluster
        group += 1             # increment the cluster counter
    else:                            # if we can continue the cluster
        counter += data['tdiff']     # increment the counter by the time difference
        hits += 1                    # increment the hits in this cluster by one
    
    return pd.Series([group,hits,counter])

Plot those histograms:

In [7]:

ROOT.enableJSVis()
c1 = ROOT.TCanvas("my1","The Canvas Title",500,500)
hist_posA.Draw("colz")
hist_posA.GetXaxis().SetTitle("mean x of cluster")
hist_posA.GetYaxis().SetTitle("mean y of cluster")
c1.Draw()

c2 = ROOT.TCanvas("my2","Title",500,500)
hist_posACoin.Draw("colz")
hist_posACoin.GetXaxis().SetTitle("mean x of cluster in coincidence")
hist_posACoin.GetYaxis().SetTitle("mean y of cluster in coincidence")
c2.Draw()

#c3 = ROOT.TCanvas("my3","Title",500,500)
#hist_timeA.Draw("colz")
#hist_timeA.GetXaxis().SetTitle("time of first hit in cluster")
#c3.Draw()

c4 = ROOT.TCanvas("my4","Title",500,500)
hist_tdiff.Draw("colz")
hist_tdiff.GetXaxis().SetTitle("time diff between trigger and timepix A")
c4.Draw()
